In [ ]:
import requests

query = """


"""

In [ ]:
def give_user_scores(user: str, type: str = "ANIME"):
    query = """
query ($search: String, $type: MediaType) { # Define which variables will be used (id)
MediaListCollection (userName: $search, type: $type) { # The sort param was POPULARITY_DESC
    lists {
        entries {
            score (format: POINT_10_DECIMAL)
            media {
                id
                title {
                    romaji
                }
            }
        }
    }
}
}

"""

    variables = {"search": user, "type": type}

    req =  requests.post(
    "https://graphql.anilist.co",
    json={"query": query, "variables": variables},
    timeout=10,
    ).json()
    scores = []
    for lists in req['data']['MediaListCollection']['lists']:
        for entry in lists['entries']:
            if entry['score'] > 0.1:
                scores.append([entry['media']['id'], entry['media']['title'], entry['score']])
    # print(scores)
    return scores


In [ ]:
import requests


In [ ]:

users = [""]
final_scores = []
for user in users:
    final_scores += give_user_scores(user,"MANGA")

final_scores


In [ ]:
from collections import Counter

# Sample list of lists
data = final_scores

# Use Counter to count the occurrences of the first entries
counts = Counter(entry[0] for entry in data)

# Create a dictionary to store the sum and count for each first entry
averages = {}

# Iterate over the data and update the sums and counts in the dictionary
for entry in data:
    key = entry[0]
    name = entry[1]
    value = entry[2]
    if key not in averages:
        averages[key] = {'sum': value, 'count': 1, 'name': name}
    else:
        averages[key]['sum'] += value
        averages[key]['count'] += 1

# Create a new list with averaged values and counters
result = []
print(averages)
for key in averages:
    average_value = averages[key]['sum'] / averages[key]['count']
    result.append([key, averages[key]['name'], average_value, counts[key]])

# Print the result
result

In [ ]:
def sort_list(e):
    return e[2]

result.sort(key=sort_list)
result

In [ ]:
real_final_list = []

for item in result:
    if item[3]>= 3:
        real_final_list.append(item)
    
def final_sorter(e):
    return e[2]

real_final_list.sort(key=final_sorter, reverse=True)
real_final_list

In [ ]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import io

In [ ]:
def id_to_image(ids: int) -> Image:
    query = """
query ($search: Int) { # Define which variables will be used (id)
Media (id: $search, type: MANGA) { # The sort param was POPULARITY_DESC
    coverImage {
        large
    }
    }
}
"""

    variables = {"search": ids}

    req =  requests.post(
    "https://graphql.anilist.co",
    json={"query": query, "variables": variables},
    timeout=10,
    ).json()
    a = Image.open(io.BytesIO(requests.get(req['data']['Media']['coverImage']['large']).content))
    # a.show()
    return a


In [ ]:

final_images = []

for item in real_final_list[:9]:
    final_images.append(id_to_image(item[0]))

In [ ]:
def make3x3(images, annotations):
    # timeInit = time.time()
    _3x3 = Image.new("RGB", (900,900), (255,255,255))
    draw = ImageDraw.Draw(_3x3) # Danger code
    # draw.text(shadow_position, annotation, font=font, fill=shadow_color)
        # draw.text((x, y), annotation, , fill=(255, 255, 255))
    font = ImageFont.truetype("arial.ttf", 20)

    for i,img in enumerate(images):
        # j = i+1
        x = (i%3)*300
        # if(i%3 == 0):
        #     x = 0
        # elif(i%3 == 1):
        #     x = 300
        # else: x = 600 

        y = int(i/3)*300
            
        # if(i < 3):
        #     y = 0
        # elif(i < 6):
        #     y = 300
        # else: y = 600
        
        # print("I is: ", i)
        size = width, height = img.size
        sizetuple = (0, 0, width, width) if height > width else (0, 0, height, height)
        # print("Size tuple: ", sizetuple)
        
        '''
        TODO 
        If the image is wider, crop centre-wise. For taller ones, height-wise is fine
        '''
        img = img.crop((0, 0, width, width) if height >= width else ((width-height)/2, 0, (width+height)/2, height)).resize((300,300))
        # print(img.size)
        # img.show()
        
        _3x3.paste(img, (x,y))
        draw.text((x, y), annotations[i], font=font, fill=(67, 64, 138)) #Danger code
        # _3x3.show()
    # print("time: ", time.time()-timeInit)
    return _3x3

In [ ]:
annotations = []
for item in real_final_list[:9]:
    annotations.append(item[2])

In [ ]:
new = []
for ratings_dec in annotations:
    # ratings_dec = "{:.2f}".format(ratings_dec)
    new.append("{:.2f}".format(ratings_dec))

In [ ]:
final_images_3x3 = make3x3(final_images, new)
final_images_3x3

In [ ]:

# annotations